In [ ]:
import json
import numpy as np
import re

In [ ]:
# Calculate data rate for single channel, single sample
def cal_RATE_one_sample_one_channel(channel, tx_power, noise):
    diag_ch = np.diag(channel)
    inter_ch = channel - np.diag(diag_ch)
    tot_ch = np.multiply(channel, np.expand_dims(tx_power, -1))
    int_ch = np.multiply(inter_ch, np.expand_dims(tx_power, -1))
    sig_ch = np.sum(tot_ch - int_ch, axis=1)
    int_ch = np.sum(int_ch, axis=1)
    SINR_val = np.divide(sig_ch, int_ch + noise)
    cap_val = np.log2(1.0 + SINR_val)
    return cap_val

# Calculate spectral and energy efficiencies
def cal_SE_EE(channel, tx_max, noise, DUE_thr, I_thr, P_c, tx_power_mat, opt="EE"):
    num_channel = 1
    num_D2D_user = channel.shape[0] - 1
    tot_SE = 0

    cur_cap = 0
    DUE_mask = 1
    CUE_mask = 1

    tx_power = np.vstack((tx_power_mat, 0 * np.ones((1, 1))))
    tx_power = np.expand_dims(tx_power, 0)

    cur_ch = channel
    cur_ch_cap = cal_RATE_one_sample_one_channel(cur_ch, tx_power[:, :, 0], noise)
    cur_cap = cur_cap + cur_ch_cap

    sum_D2D_SE_temp = np.sum(cur_cap[0, :-1])
    sum_D2D_EE_temp = np.sum(cur_cap[0, :-1] / (tx_power[0, :-1, 0] + P_c))

    D2D_SE_sum = sum_D2D_SE_temp
    D2D_EE_sum = sum_D2D_EE_temp

    return D2D_SE_sum, D2D_EE_sum


In [ ]:
def process_predicted_response(predicted_response):
    """
    This function processes the predicted response to ensure that:
    - If there are no values, it returns [0, 0].
    - If there is 1 value, it appends a 0.
    - If there are more than 2 values, it takes only the first 2.
    - Otherwise, it keeps the values as they are.
    """
    # Check if the response is empty or has no valid values
    if not predicted_response or predicted_response.strip() == "":
        return [0, 0]

    # Convert the string response to an array of values
    numbers = [int(num) for num in re.findall(r'-?\d+', predicted_response)]

    # Filter for non-negative numbers and take the first two
    non_negative_numbers = [num for num in numbers if num >= 0][:2]

    # If there are less than 2 values, pad with zeros
    while len(non_negative_numbers) < 2:
        non_negative_numbers.append(0)

    return non_negative_numbers

In [ ]:
# Function to create DPO training dataset from input dataset
def create_dpo_dataset_from_json(input_file, ee_val_info_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)

    # Load additional information for the validation dataset
    with open(ee_val_info_file, "r") as f:
        ee_val_info = json.load(f)

    # Initialize list to store DPO dataset entries
    dpo_dataset = []

    # Create a dictionary for quick access based on sample_index
    ee_val_info_dict = {record["sample_index"]: record for record in ee_val_info}

    for example in data:
      efficiencies = []
      sample_index = int(example["sample_index"])

      # Get the original information from the corresponding dataset
      if sample_index in ee_val_info_dict:
          original_info = ee_val_info_dict[sample_index]
      else:
          print(f"Sample index {sample_index} not found in any dataset.")
          continue

      # Extract necessary fields
      channel = np.array(original_info["chan_mat_values"])
      tx_max = original_info["tx_max"]
      noise = original_info["noise"]
      DUE_thr = original_info["DUE_thr"]
      I_thr = original_info["I_thr"]
      P_c = original_info["P_c"]
      original_power = np.array(original_info["pw_vec_values"]).flatten().reshape(2, 1)

      # Extract responses
      original_response_float = [float(x) for x in example['original_response'].split(',')]
      original_response = [int(x) for x in original_response_float]

      # Process predicted responses
      predicted_power_1 = process_predicted_response(example["predicted_response_1"])
      predicted_power_2 = process_predicted_response(example["predicted_response_2"])
      predicted_power_3 = process_predicted_response(example["predicted_response_3"])
      predicted_responses = [predicted_power_1, predicted_power_2, predicted_power_3]

      # Calculate energy efficiency for all responses
      _, predicted_EE_1 = cal_SE_EE(channel, tx_max, noise, DUE_thr, I_thr, P_c, np.array(predicted_power_1).reshape(2, 1), opt=criteria)
      _, predicted_EE_2 = cal_SE_EE(channel, tx_max, noise, DUE_thr, I_thr, P_c, np.array(predicted_power_2).reshape(2, 1), opt=criteria)
      _, predicted_EE_3 = cal_SE_EE(channel, tx_max, noise, DUE_thr, I_thr, P_c, np.array(predicted_power_3).reshape(2, 1), opt=criteria)
      efficiencies = [predicted_EE_1, predicted_EE_2, predicted_EE_3]

      # Identify least preferred response (lowest efficiency)
      least_preferred_idx = efficiencies.index(min(efficiencies))
      least_preferred_response = [int(i) for i in predicted_responses[least_preferred_idx]]

      # Prepare the DPO entry
      dpo_entry = {
          "sample_index": example["sample_index"],
          "instruction": example["instruction"],
          "input": example["input"],
          "most_preferred_response": (",".join(map(str, original_response)))+".",
          "least_preferred_response": (",".join(map(str, least_preferred_response))+"."),
      }

      # Append to the dataset
      dpo_dataset.append(dpo_entry)

    # Save the DPO dataset as a JSON file
    with open(output_file, 'w') as f:
        json.dump(dpo_dataset, f, indent=4)

In [ ]:
input_file = 'generated_responses_ee_with_three_responses.json'  # Input file in JSON format
ee_val_info_file = "ee_val_data.json"
output_file = 'dpo_dataset.json'   # Desired output file

criteria = "EE"

create_dpo_dataset_from_json(input_file, ee_val_info_file, output_file)
print(f"DPO dataset saved to {output_file}")

DPO dataset saved to dpo_dataset.json
